# Исследование надежности заемщиков

# Задача

На основе статистики о платёжеспособности клиентов исследовать влияет ли семейное положение и количество детей клиента на факт возврата кредита в срок.

# План

**1. Проведение обзора данных.**

**2. Проведение предобработки данных.**

**3. Проверить, есть ли зависимость между количеством детей и возвратом кредита в срок.**

**4. Проверить, есть ли зависимость между семейным положением и возвратом кредита в срок.**

**5. Проверить, есть ли зависимость между уровнем дохода и возвратом кредита в срок.**

**6. Проверить, как разные цели кредита влияют на его возврат в срок.**

**7. Выводы.**

## Обзор данных

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('/datasets/data.csv')

In [3]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

## Предобработка данных

### Удаление пропусков

In [5]:
# Подсчет пропусков
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Столбец total_income хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполним пропуски в этом столбце медианным значением по каждому типу из столбца income_type.

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()
data['total_income'].isna().sum()

0

### Обработка аномальных значений

В столбце days_employed встречается отрицательное количество дней трудового стажа. Заменим все отрицательные значения положительными с помощью метода abs().

In [7]:
data['days_employed'] = data['days_employed'].abs()
data['days_employed']

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64

Для каждого типа занятости проверим медианное значение трудового стажа days_employed в днях.

In [8]:
data.groupby('income_type')['days_employed'].median()

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть, но будем иметь ввиду при поведении исследования.

In [9]:
# проверим столбец children
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце children есть два аномальных значения: -1 и 20. Удалим строки, в которых они встречаются.

In [10]:
data = data.loc[(data['children'] != -1) & (data['children'] != 20)]

In [11]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

In [12]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()
data['days_employed'].isna().sum()

0

In [13]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

От пропусков и аномальных значений избавились.

### Изменение типов данных

In [14]:
# приведем столбц total_income к целочисленному типу
data['total_income'] = data['total_income'].astype('int')

### Обработка дубликатов

In [15]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Одно и то же образование написано в нескольких вариантах. Приведем все названия к нижнему регистру.

In [16]:
data['education'] = data['education'].str.lower()

In [17]:
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [18]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [19]:
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [20]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [21]:
# Считаем кол-во явных дубликатов
data.duplicated().sum()

71

In [22]:
data = data.drop_duplicates()
data.duplicated().sum()

0

### Категоризация данных

Нам предложено на основании диапазонов, указанных ниже, добавить столбец с категориями дохода:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [23]:
def categorize_income(income):
    if income <= 30000:
        return 'E'
    if 30001 <= income <= 50000:
        return 'D'
    if 50001 <= income <= 200000:
        return 'C'
    if 200001 <= income <= 1000000:
        return 'B'
    return 'A'

In [24]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Также предложено на основании столбца purpose сформировать новый столбец, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [25]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [26]:
def categorize_purpose(purpose):
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if ('жиль' in purpose or 'недвижимост' in purpose):
        return 'операции с недвижимостью'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образован' in purpose:
        return 'получение образования'
    return 'неопределен'

In [27]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Проверим, есть ли зависимость между количеством детей и возвратом кредита в срок.

In [28]:
#Построим сводную таблицу по количеству детей и посчитаем процент должников
def debt_type(debt):
    if debt == 0:
        return 'no debt'
    return 'debt'

data['count_borrower'] = 'count_borrower'
data['debt_type'] = data['debt'].apply(debt_type)
data_pivot1 = data.pivot_table(index='children', columns='count_borrower', values= 'debt', aggfunc='count')
data_pivot2 = data.pivot_table(index='children', columns='debt_type', values= 'debt', aggfunc='count')
data_pivot = data_pivot1.merge(data_pivot2, on='children', how='left')
data_pivot['ratio'] = data_pivot['debt'] / data_pivot['count_borrower'] * 100
data_pivot

,count_borrower,debt,no debt,ratio
children,,,,
0,14091,1063.0,13028.0,7.543822
1,4808,444.0,4364.0,9.234609
2,2052,194.0,1858.0,9.454191
3,330,27.0,303.0,8.181818
4,41,4.0,37.0,9.756098
5,9,NaN,9.0,NaN


**Вывод:**

Больше всего заемщиков с количеством детей 0, 1 и 2. В категории '0' заемщиков в два разва больше, чем в категориях '1' и '2' вместе взятых. Процент должников в категории '0' составляет 7,5%, в категории '1' - 9,2%, в категории '2' - 9,5%. Т.е. чем больше у заемщика детей, тем выше риск допустить просрочку.

## Проверим, есть ли зависимость между семейным положением и возвратом кредита в срок.

In [29]:
#Построим сводную таблицу по семейному положению и посчитаем процент должников
data_pivot_ststus1 = data.pivot_table(index='family_status', columns='count_borrower', values= 'debt', aggfunc='count')
data_pivot_ststus2 = data.pivot_table(index='family_status', columns='debt_type', values= 'debt', aggfunc='count')
data_pivot_ststus = data_pivot_ststus1.merge(data_pivot_ststus2, on='family_status', how='left')
data_pivot_ststus['ratio'] = data_pivot_ststus['debt'] / data_pivot_ststus['count_borrower'] * 100
data_pivot_ststus

,count_borrower,debt,no debt,ratio
family_status,,,,
Не женат / не замужем,2796,273,2523,9.763948
в разводе,1189,84,1105,7.064760
вдовец / вдова,951,63,888,6.624606
гражданский брак,4134,385,3749,9.313014
женат / замужем,12261,927,11334,7.560558


**Вывод:**

Больше всего заемщиков со статусами 'женат / замужем', 'гражданский брак' и 'Не женат / не замужем'. В категории 'женат / замужем' процент должников составляет 7,6%, в категории 'гражданский брак' - 9,3%, в категории 'Не женат / не замужем' - 9,7%. У категорий 'в разводе ' и 'вдовец / вдова' число заемщиков намного меньше, но и процент должников там тоже меньше.

Итак, можно сделать вывод, что самыми рискованными категориями заемщиков являются 'гражданский брак' и 'Не женат / не замужем'. Самая многочисленная категория 'женат / замужем' является менее рискованной.

## Проверим, есть ли зависимость между уровнем дохода и возвратом кредита в срок.

In [30]:
#Построим сводную таблицу по категориям дохода и посчитаем процент должников
data_pivot_income_category1 = data.pivot_table(index='total_income_category', columns='count_borrower', values= 'debt', aggfunc='count')
data_pivot_income_category2 = data.pivot_table(index='total_income_category', columns='debt_type', values= 'debt', aggfunc='count')
data_pivot_income_category = data_pivot_income_category1.merge(data_pivot_income_category2, on='total_income_category', how='left')
data_pivot_income_category['ratio'] = data_pivot_income_category['debt'] / data_pivot_income_category['count_borrower'] * 100
data_pivot_income_category

,count_borrower,debt,no debt,ratio
total_income_category,,,,
A,25,2,23,8.000000
B,5014,354,4660,7.060231
C,15921,1353,14568,8.498210
D,349,21,328,6.017192
E,22,2,20,9.090909


**Вывод:**

Больше всего заемщиков с уровнем дохода 'С' и 'B'. В категории 'С' процент должников составляет 8,5%, а в категории 'B' - 7,1%. Следовательно, более рискованной будет категория заемщиков 'С', она же является самой многочисленной. А в категории 'B' меньше риска.

## Проверим, как разные цели кредита влияют на его возврат в срок.

In [31]:
#Построим сводную таблицу по категориям целей кредита и посчитаем процент должников
data_pivot_purpose_category1 = data.pivot_table(index='purpose_category', columns='count_borrower', values= 'debt', aggfunc='count')
data_pivot_purpose_category2 = data.pivot_table(index='purpose_category', columns='debt_type', values= 'debt', aggfunc='count')
data_pivot_purpose_category = data_pivot_purpose_category1.merge(data_pivot_purpose_category2, on='purpose_category', how='left')
data_pivot_purpose_category['ratio'] = data_pivot_purpose_category['debt'] / data_pivot_purpose_category['count_borrower'] * 100
data_pivot_purpose_category

,count_borrower,debt,no debt,ratio
purpose_category,,,,
операции с автомобилем,4279,400,3879,9.347978
операции с недвижимостью,10751,780,9971,7.255139
получение образования,3988,369,3619,9.252758
проведение свадьбы,2313,183,2130,7.911803


**Вывод:**

Самые многочисленные категории 'операции с недвижимостью', 'операции с автомобилем', 'получение образования'. Процент должников в них составляет 7,3%, 9,4% и 9,3% соответственно. Из чего можно сделать вывод, что наименее рискованной является самая многочисленная категория 'операции с недвижимостью', а категории 'операции с автомобилем', 'получение образования' будут самыми рискованными.

## Выводы

Мы получили от банка статистику о платежеспособности клиентов.

Наша цель: выяснить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

На этапе предобработки данных мы устранили аномальные значения, пропуски и дубликаты.

В ходе исследования мы выяснили:

1) Чем больше у заемщика детей, тем выше риск допустить просрочку.

2) Самыми рискованными категориями заемщиков являются 'гражданский брак' и 'Не женат / не замужем'. Самая многочисленная категория 'женат / замужем' является менее рискованной.

3) Более рискованной будет категория заемщиков 'С', она же является самой многочисленной. А в категории 'B' меньше риска.

4) Менее рискованной является самая многочисленная категория 'операции с недвижимостью', а категории 'операции с автомобилем', 'получение образования' будут самыми рискованными.